## 配置环境，生成.so文件

In [36]:
from tree_sitter import Language
Language.build_library(
  # so文件保存位置
  'build/my-languages.so',

  # vendor文件下git clone的仓库
  [
    'vendor/tree-sitter-java',
    'vendor/tree-sitter-python',
    'vendor/tree-sitter-cpp',
    'vendor/tree-sitter-c-sharp',
    'vendor/tree-sitter-javascript',
    'vendor/tree-sitter-go',
    'vendor/tree-sitter-php',
    'vendor/tree-sitter-typescript/tsx/'
  ]
)

False

## 导入treesitter库，生成各个语言的parser

In [37]:
from tree_sitter import Language, Parser
CPP_LANGUAGE = Language('build/my-languages.so', 'cpp')
PY_LANGUAGE = Language('build/my-languages.so', 'python')
JAVA_LANGUAGE=Language('build/my-languages.so', 'java')
JS_LANGUAGE=Language('build/my-languages.so', 'javascript')
C_SHARP_LANGUAGE=Language('build/my-languages.so', 'c_sharp')
GO_LANGUAGE=Language('build/my-languages.so', 'go')
PHP_LANGUAGE=Language('build/my-languages.so', 'php')
TYPESCRIPT_LANGUAGE=Language('build/my-languages.so','tsx')
#Python
py_parser = Parser()
py_parser.set_language(PY_LANGUAGE)
#Cpp
cpp_parser=Parser()
cpp_parser.set_language(CPP_LANGUAGE)
#java
java_parser=Parser()
java_parser.set_language(JAVA_LANGUAGE)
#JS
js_parser=Parser()
js_parser.set_language(JS_LANGUAGE)
#c-sharp
c_sharp_parser=Parser()
c_sharp_parser.set_language(C_SHARP_LANGUAGE)
#Go
go_parser=Parser()
go_parser.set_language(GO_LANGUAGE)
#PHP
php_parser=Parser()
php_parser.set_language(PHP_LANGUAGE)
#Typescript
typescript_parser=Parser()
typescript_parser.set_language(TYPESCRIPT_LANGUAGE)


## 生成嵌套列表格式的ast
函数process_string(text,left='[[]',right='[]]',sep=','),将lisp风格的括号嵌套字符串转化为python列表。
exp:(module(indicator))=>['module',['indicator']]
函数code2ast(language:string,filepath:string)
exp:
"examples/example_python.py"

In [38]:
import re
def process_string(text,left='[[]',right='[]]',sep=' '):
    pat = r'({}|{}|{})'.format(left, right, sep)
    tokens = re.split(pat, text)
    stack = [[]]
    for x in tokens:
        if not x or re.match(sep, x): 
            continue
        if re.match(left, x):
            stack[-1].append([])
            stack.append(stack[-1][-1])
        elif re.match(right, x):
            stack.pop()
            if not stack:
                return 'error: opening bracket is missing'
                #raise ValueError('error: opening bracket is missing')
        else:
            stack[-1].append(x)
    if len(stack) > 1:
        return 'error: closing bracket is missing'
        raise ValueError('error: closing bracket is missing')
    return stack.pop()

In [39]:

def code2ast(language:str,filepath:str):
    with open(filepath, "r") as f:
        code = f.read()
    #生成parsetree
    if language=="Python":
        tree = py_parser.parse(bytes(code, "utf8"))
    elif language=="Cpp":
        tree=cpp_parser.parse(bytes(code,"utf8"))
    elif language=="java":
        tree=java_parser.parse(bytes(code,"utf8"))
    elif language=="javascript":
        tree=js_parser.parse(bytes(code,"utf8"))
    elif language=="C-sharp":
        tree=c_sharp_parser.parse(bytes(code,"utf8"))
    elif language=="Go":
        tree=go_parser.parse(bytes(code,"utf8"))
    elif language=="PHP":
        tree=php_parser.parse(bytes(code,"utf8"))
    elif language=="typescript":
        tree=typescript_parser.parse(bytes(code,"utf8"))
    root_node=tree.root_node
    string=root_node.sexp()
    string=string.replace('(','[')
    string=string.replace(')',']')
    return process_string(string)

### 测试test

In [40]:
print("Cpp")
print(code2ast("Cpp","examples/example_cpp.cpp"))
print("\n")
print("Python")
print(code2ast("Python","examples/example_python.py"))
print("\n")
#print("java")
#print(code2ast("java","examples/example_java.java"))
#print("\n")
#print("JS")
#print(code2ast("javascript","examples/example_js.js"))
#print("\n")
#print("Go")
#print(code2ast("Go","examples/example_go.go"))
#print("\n")
#print("php")
#print(code2ast("PHP","examples/example_php.php"))
#print("\n")
#print("C-sharp")
#print(code2ast("C-sharp","examples/example_c_sharp.cs"))
#print("\n")
#print("TS")
#print(code2ast("typescript","examples/example_ts.ts"))
#print("\n")


Cpp
[['translation_unit', ['function_definition', 'type:', ['primitive_type'], 'declarator:', ['function_declarator', 'declarator:', ['identifier'], 'parameters:', ['parameter_list', ['parameter_declaration', 'type:', ['primitive_type'], 'declarator:', ['identifier']]]], 'body:', ['compound_statement', ['declaration', 'type:', ['primitive_type'], 'declarator:', ['init_declarator', 'declarator:', ['identifier'], 'value:', ['number_literal']]], ['for_statement', 'initializer:', ['declaration', 'type:', ['primitive_type'], 'declarator:', ['init_declarator', 'declarator:', ['identifier'], 'value:', ['number_literal']]], 'condition:', ['binary_expression', 'left:', ['identifier'], 'right:', ['identifier']], 'update:', ['update_expression', 'argument:', ['identifier']], 'body:', ['compound_statement', ['expression_statement', ['assignment_expression', 'left:', ['identifier'], 'right:', ['identifier']]]]], ['return_statement', ['identifier']]]]]]


Python
[['module', ['function_definition', '